In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/raw/green/*/*')

### Expressing SQL  statement in RDDs terms

```sql
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [6]:
rows = rdd.take(10)
row = rows[0]

In [7]:
row

Row(lpep_pickup_datetime=datetime.datetime(2019, 12, 18, 18, 52, 30), PULocationID=264, total_amount=4.81)

In [8]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [9]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [10]:
from collections import namedtuple

In [11]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [12]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [13]:
from pyspark.sql import types

In [14]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [15]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [16]:
df_result.write.parquet('tmp/green-revenue')

In [17]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [18]:
import pandas as pd

In [19]:
rows = duration_rdd.take(10)

In [21]:
df = pd.DataFrame(rows, columns=columns)
df

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2,2019-12-18 18:52:30,264,264,0.00
1,2,2020-01-01 03:45:58,66,65,1.28
2,2,2020-01-01 03:41:38,181,228,2.47
3,1,2020-01-01 03:52:46,129,263,6.30
4,1,2020-01-01 03:19:57,210,150,2.30
5,1,2020-01-01 03:52:33,35,39,3.00
6,2,2020-01-01 03:10:18,25,61,2.77
7,2,2020-01-01 04:03:14,225,89,4.98
8,2,2020-01-01 03:04:11,129,129,0.71
9,2,2020-01-01 03:25:52,129,83,0.80


In [22]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [23]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [24]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [25]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [26]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|               0.0|
|               6.4|
|12.350000000000001|
|              31.5|
|              11.5|
|              15.0|
|             13.85|
|24.900000000000002|
|              3.55|
|               4.0|
|               7.6|
|              19.0|
|5.6000000000000005|
|             28.35|
|               9.3|
|               7.1|
|              77.4|
|              5.75|
|              11.5|
|               5.0|
+------------------+
only showing top 20 rows

